In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


/Users/JCReynoldsMacbook/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train_data = pd.read_csv("./titanic/train.csv")
test_data = pd.read_csv("./titanic/test.csv")


In [3]:
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())
train_data1 = train_data.drop(["Cabin", "Name"], axis=1)
test_data1 = test_data.drop(["Cabin", "Name"], axis=1)

In [5]:
import re

train_data2 = train_data1.join(pd.get_dummies(train_data1['Sex'], prefix='', prefix_sep=''))
train_data2 = train_data2.drop(["Sex"], axis=1)
train_data2["Ticket"] = train_data2["Ticket"].str.replace(r'[^0-9]+', "0")
train_data2["Ticket"] = train_data2["Ticket"].astype(int)


In [6]:
test_data2 = test_data1.join(pd.get_dummies(test_data1['Sex'], prefix='', prefix_sep=''))
test_data2 = test_data2.drop(["Sex"], axis=1)
test_data2["Ticket"] = test_data2["Ticket"].str.replace(r'[^0-9]+', '')
test_data2["Ticket"] = test_data2["Ticket"].astype(int)


In [7]:
train_data2["totalfam"] = train_data2["SibSp"] + train_data2["Parch"]
test_data2["totalfam"] = test_data2["SibSp"] + test_data2["Parch"]

In [11]:
train_data2['Embarked'] = train_data2['Embarked'].fillna('S').map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

test_data2['Embarked'] = test_data2['Embarked'].fillna('S').map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_data2.tail()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Embarked,female,male,totalfam
413,1305,3,27.0,0,0,53236,8.0500,0,0,1,0
414,1306,1,39.0,0,0,17758,108.9000,1,1,0,0
415,1307,3,38.5,0,0,3101262,7.2500,0,0,1,0
416,1308,3,27.0,0,0,359309,8.0500,0,0,1,0
417,1309,3,27.0,1,1,2668,22.3583,1,0,1,2


In [12]:
train_data2.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Embarked,female,male,totalfam
0,1,0,3,22.0,1,0,5021171,7.2500,0,0,1,1
1,2,1,1,38.0,1,0,17599,71.2833,1,1,0,1
2,3,1,3,26.0,0,0,203101282,7.9250,0,1,0,0
3,4,1,1,35.0,1,0,113803,53.1000,0,1,0,1
4,5,0,3,35.0,0,0,373450,8.0500,0,0,1,0


In [15]:
test_data2["Fare"] = test_data2["Fare"].median()

In [14]:
train_data2.info()
test_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null int64
Fare           891 non-null float64
Embarked       891 non-null int64
female         891 non-null uint8
male           891 non-null uint8
totalfam       891 non-null int64
dtypes: float64(2), int64(8), uint8(2)
memory usage: 71.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null int64
Fare           417 non-null float64
Embarked       418 non-null int64
female         418 non-null uint8
male   

In [30]:
x_train = train_data2.drop(["Survived", "PassengerId"], axis=1)
y_train = train_data2["Survived"]
x_test  = test_data2.drop("PassengerId", axis=1).copy()
x_train.shape, y_train.shape, x_test.shape
x_train.head()

,Pclass,Age,SibSp,Parch,Ticket,Fare,Embarked,female,male,totalfam
0,3,22.0,1,0,5021171,7.2500,0,0,1,1
1,1,38.0,1,0,17599,71.2833,1,1,0,1
2,3,26.0,0,0,203101282,7.9250,0,1,0,0
3,1,35.0,1,0,113803,53.1000,0,1,0,1
4,3,35.0,0,0,373450,8.0500,0,0,1,0


In [144]:
x_test.head()

,Pclass,Age,SibSp,Parch,Ticket,Fare,Embarked,female,male,totalfam
0,3,34.5,0,0,330911,14.4542,3,0,1,0
1,3,47.0,1,0,363272,14.4542,1,1,0,1
2,2,62.0,0,0,240276,14.4542,3,0,1,0
3,3,27.0,0,0,315154,14.4542,1,0,1,0
4,3,22.0,1,1,3101298,14.4542,1,1,0,2


In [31]:
train_data2.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Ticket           int64
Fare           float64
Embarked         int64
female           uint8
male             uint8
totalfam         int64
dtype: object

In [32]:
x_train.shape, y_train.shape

((891, 10), (891,))

In [33]:
x_test.shape

(418, 10)

In [36]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_train, y_train) * 100, 2)
acc_log

61.62

In [41]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree

99.78

In [47]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
Y_pred = random_forest.predict(x_test)
random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
acc_random_forest

99.78

In [43]:
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,